# Best 100-epoch full pipeline (Audio → Video → Fusion)

This notebook prepares a dedicated config and runs three training stages in sequence:
1. Audio (MIL multiclass) for 100 epochs
2. Video window model for 100 epochs
3. Temporal fusion for 100 epochs using the best audio+video checkpoints produced above

In [1]:
from pathlib import Path
import json
import os
import subprocess
import sys

ROOT = Path.cwd()
if not (ROOT / "configs" / "master_config.json").exists():
    raise RuntimeError(f"Run this notebook from repo root. Current dir: {ROOT}")

print(f"Repo root: {ROOT}")
print(f"Python: {sys.executable}")

Repo root: /home/alolli/src/malto/hackathon/therness-hackaton-2026-polito
Python: /home/alolli/miniconda3/envs/therness_env/bin/python


In [ ]:
base_cfg_path = ROOT / "configs" / "master_config.json"
run_cfg_path = ROOT / "configs" / "master_config.best_100_pipeline.json"

cfg = json.loads(base_cfg_path.read_text())

data_root_candidates = [
    Path("/data1/malto/therness/data/Hackathon"),
    Path("/root/data/train_data"),
    ROOT / "sampleData",
]
resolved_data_root = None
for candidate in data_root_candidates:
    if candidate.exists() and any(candidate.rglob("*.flac")):
        resolved_data_root = str(candidate)
        break
if resolved_data_root is None:
    raise FileNotFoundError(
        "No valid data root found. Checked: "
        + ", ".join(str(p) for p in data_root_candidates)
    )
cfg["train_data_root"] = resolved_data_root
cfg["data_root"] = resolved_data_root
print(f"Using data_root: {resolved_data_root}")

# ---- Target metric threshold: stop early if metric >= this ----
TARGET_METRIC_THRESHOLD = 0.95

# ---- AUDIO: best-known multiclass MIL config, 100 epochs ----
ac = cfg.setdefault("audio", {})
am = ac.setdefault("model", {})
at = ac.setdefault("training", {})
amil = at.setdefault("sequence_mil", {})

am["dropout"] = 0.15
at["task"] = "multiclass"
at["num_epochs"] = 100
at["lr"] = 7e-05
at["weight_decay"] = 3e-05
at["patience"] = 30
at["checkpoint_dir"] = "checkpoints/audio_best_100"
at["metric"] = "macro_f1"
at["target_metric_threshold"] = TARGET_METRIC_THRESHOLD

amil["enabled"] = True
amil["batch_size"] = 8
amil["topk_ratio_pos"] = 0.12
amil["topk_ratio_neg"] = 0.2
amil["eval_pool_ratio"] = 0.12
amil["auto_threshold"] = True
amil["good_window_weight"] = 0.0
amil["use_class_weights"] = True
amil["class_weight_power"] = 0.65
amil["multiclass_eval_mode"] = "topk_per_class"
amil["use_balanced_sampler"] = True
amil["balanced_sampler_power"] = 0.35

# ---- VIDEO: best-known window config, 100 epochs ----
vw = cfg.setdefault("video_window", {})
vwm = vw.setdefault("model", {})
vwt = vw.setdefault("training", {})

vwm["dropout"] = 0.12
vwt["epochs"] = 100
vwt["lr"] = 1e-4
vwt["weight_decay"] = 7e-5
vwt["class_weight_power"] = 1.0
vwt["use_balanced_sampler"] = True
vwt["balanced_sampler_power"] = 0.3
vwt["patience"] = 30
vwt["checkpoint_dir"] = "checkpoints/video_best_100"
vwt["checkpoint_path"] = "checkpoints/video_best_100/window_classifier.pth"
vwt["target_metric_threshold"] = TARGET_METRIC_THRESHOLD

# ---- FUSION: temporal config, 100 epochs ----
fc = cfg.setdefault("fusion", {})
fm = fc.setdefault("model", {})
ft = fc.setdefault("training", {})

fm["arch"] = "temporal"
fm["audio_dim"] = 128
fm["video_dim"] = 128
fm["hidden_dim"] = 192
fm["dropout"] = 0.2
fm["sequence_len"] = 12
fm["temporal_layers"] = 1

ft["num_epochs"] = 100
ft["lr"] = 1e-4
ft["weight_decay"] = 1e-4
ft["batch_size"] = 64
ft["patience"] = 30
ft["checkpoint_dir"] = "checkpoints/fusion_best_100"
ft["target_metric_threshold"] = TARGET_METRIC_THRESHOLD

run_cfg_path.write_text(json.dumps(cfg, indent=2))
print(f"Wrote config: {run_cfg_path}")
print(f"Target metric threshold: {TARGET_METRIC_THRESHOLD} (all models)")

Wrote config: /home/alolli/src/malto/hackathon/therness-hackaton-2026-polito/configs/master_config.best_100_pipeline.json


In [3]:
def run_cmd(cmd):
    print('\n' + '='*90)
    print('RUN:', ' '.join(cmd))
    print('='*90)
    subprocess.run(cmd, check=True, cwd=ROOT)

audio_ckpt = ROOT / "checkpoints" / "audio_best_100" / "best_model.pt"
video_ckpt = ROOT / "checkpoints" / "video_best_100" / "best_model.pt"
fusion_ckpt = ROOT / "checkpoints" / "fusion_best_100" / "best_model.pt"

print('Expected outputs:')
print(' -', audio_ckpt)
print(' -', video_ckpt)
print(' -', fusion_ckpt)

Expected outputs:
 - /home/alolli/src/malto/hackathon/therness-hackaton-2026-polito/checkpoints/audio_best_100/best_model.pt
 - /home/alolli/src/malto/hackathon/therness-hackaton-2026-polito/checkpoints/video_best_100/best_model.pt
 - /home/alolli/src/malto/hackathon/therness-hackaton-2026-polito/checkpoints/fusion_best_100/best_model.pt


## 0) Hyperparameter tuning (short trials before full training)

Run 15-epoch trials with random search to find the best hyperparameters for each model.
Results are stored in `tuning_results` dict and auto-applied to the full 100-epoch config.

In [ ]:
import copy, random as _rnd, shutil, time, torch

TUNING_EPOCHS = 15          # short trials
TUNING_PATIENCE = 8         # early stop within each trial
N_AUDIO_TRIALS = 6
N_VIDEO_TRIALS = 6
N_FUSION_TRIALS = 6

tuning_results = {"audio": [], "video": [], "fusion": []}

def _sample_uniform(lo, hi, log=False):
    """Sample from [lo, hi] uniformly (or log-uniformly)."""
    if log:
        import math
        return math.exp(_rnd.uniform(math.log(lo), math.log(hi)))
    return _rnd.uniform(lo, hi)

def _sample_choice(options):
    return _rnd.choice(options)

def run_trial(module, config_overrides, trial_cfg, trial_name, extra_args=None):
    """Run a short training trial and return the best metric from checkpoint."""
    trial_dir = f"checkpoints/_tuning/{trial_name}"
    trial_cfg_path = ROOT / "configs" / f"_tuning_{trial_name}.json"

    # Deep copy & apply overrides
    tcfg = copy.deepcopy(cfg)
    tcfg["data_root"] = cfg["data_root"]
    tcfg["train_data_root"] = cfg.get("train_data_root", cfg["data_root"])

    for key_path, value in config_overrides.items():
        parts = key_path.split(".")
        d = tcfg
        for p in parts[:-1]:
            d = d.setdefault(p, {})
        d[parts[-1]] = value

    trial_cfg_path.write_text(json.dumps(tcfg, indent=2))

    cmd = [sys.executable, '-u', '-m', module, '--config', str(trial_cfg_path)]
    if extra_args:
        cmd.extend(extra_args)

    print(f"\n{'─'*70}")
    print(f"TRIAL: {trial_name}")
    print(f"  Params: {trial_cfg}")
    print(f"{'─'*70}")
    t0 = time.time()

    try:
        subprocess.run(cmd, check=True, cwd=ROOT)
    except subprocess.CalledProcessError:
        print(f"  TRIAL FAILED: {trial_name}")
        trial_cfg_path.unlink(missing_ok=True)
        return None

    elapsed = time.time() - t0
    best_ckpt = ROOT / trial_dir / "best_model.pt"
    if not best_ckpt.exists():
        print(f"  No best checkpoint produced for {trial_name}")
        trial_cfg_path.unlink(missing_ok=True)
        return None

    ckpt = torch.load(best_ckpt, map_location="cpu")
    # Audio uses val_f1, video/fusion use hackathon_score
    score = ckpt.get("hackathon_score", ckpt.get("val_f1", 0.0))
    epoch = ckpt.get("epoch", "?")
    print(f"  RESULT: score={score:.4f}  epoch={epoch}  time={elapsed:.0f}s")

    # Clean up trial config file
    trial_cfg_path.unlink(missing_ok=True)

    return {"name": trial_name, "score": float(score), "epoch": epoch,
            "params": trial_cfg, "elapsed": elapsed}

print(f"Tuning config: {TUNING_EPOCHS} epochs, patience {TUNING_PATIENCE}")
print(f"Trials: audio={N_AUDIO_TRIALS}, video={N_VIDEO_TRIALS}, fusion={N_FUSION_TRIALS}")

### 0a) Audio tuning

In [ ]:
_rnd.seed(42)

for i in range(N_AUDIO_TRIALS):
    trial_params = {
        "lr":           _sample_uniform(4e-5, 1e-4, log=True),
        "weight_decay": _sample_uniform(1e-5, 8e-5, log=True),
        "dropout":      _sample_choice([0.10, 0.12, 0.15, 0.18, 0.20]),
        "cwp":          _sample_uniform(0.5, 0.8),
        "bsp":          _sample_uniform(0.25, 0.45),
        "topk_pos":     _sample_choice([0.08, 0.10, 0.12, 0.15]),
    }
    trial_name = (
        f"audio_tune_{i:02d}_lr={trial_params['lr']:.1e}"
        f"_wd={trial_params['weight_decay']:.1e}"
        f"_do={trial_params['dropout']}"
    )
    overrides = {
        "audio.training.num_epochs":                        TUNING_EPOCHS,
        "audio.training.patience":                          TUNING_PATIENCE,
        "audio.training.lr":                                trial_params["lr"],
        "audio.training.weight_decay":                      trial_params["weight_decay"],
        "audio.model.dropout":                              trial_params["dropout"],
        "audio.training.sequence_mil.class_weight_power":   trial_params["cwp"],
        "audio.training.sequence_mil.balanced_sampler_power": trial_params["bsp"],
        "audio.training.sequence_mil.topk_ratio_pos":       trial_params["topk_pos"],
        "audio.training.sequence_mil.eval_pool_ratio":      trial_params["topk_pos"],
        "audio.training.checkpoint_dir":                    f"checkpoints/_tuning/{trial_name}",
    }
    result = run_trial("audio.run_audio", overrides, trial_params, trial_name)
    if result:
        tuning_results["audio"].append(result)

# Sort and display
tuning_results["audio"].sort(key=lambda r: r["score"], reverse=True)
print(f"\n{'='*70}")
print("AUDIO TUNING RESULTS (sorted by score):")
print(f"{'='*70}")
for r in tuning_results["audio"]:
    print(f"  {r['score']:.4f}  ep={r['epoch']}  {r['name']}  params={r['params']}")

if tuning_results["audio"]:
    best_audio = tuning_results["audio"][0]
    print(f"\nBEST AUDIO: {best_audio['score']:.4f} — {best_audio['params']}")
else:
    print("\nNo successful audio trials")

### 0b) Video tuning

In [ ]:
_rnd.seed(123)

for i in range(N_VIDEO_TRIALS):
    trial_params = {
        "lr":           _sample_uniform(7e-5, 1.5e-4, log=True),
        "weight_decay": _sample_uniform(3e-5, 1.5e-4, log=True),
        "dropout":      _sample_choice([0.10, 0.12, 0.14, 0.15, 0.18]),
        "cwp":          _sample_uniform(0.85, 1.15),
        "bsp":          _sample_uniform(0.2, 0.45),
    }
    trial_name = (
        f"video_tune_{i:02d}_lr={trial_params['lr']:.1e}"
        f"_wd={trial_params['weight_decay']:.1e}"
        f"_do={trial_params['dropout']}"
    )
    overrides = {
        "video_window.training.epochs":                 TUNING_EPOCHS,
        "video_window.training.patience":               TUNING_PATIENCE,
        "video_window.training.lr":                     trial_params["lr"],
        "video_window.training.weight_decay":           trial_params["weight_decay"],
        "video_window.model.dropout":                   trial_params["dropout"],
        "video_window.training.class_weight_power":     trial_params["cwp"],
        "video_window.training.balanced_sampler_power": trial_params["bsp"],
        "video_window.training.checkpoint_dir":         f"checkpoints/_tuning/{trial_name}",
        "video_window.training.checkpoint_path":        f"checkpoints/_tuning/{trial_name}/window_classifier.pth",
    }
    result = run_trial("video.run_video", overrides, trial_params, trial_name)
    if result:
        tuning_results["video"].append(result)

tuning_results["video"].sort(key=lambda r: r["score"], reverse=True)
print(f"\n{'='*70}")
print("VIDEO TUNING RESULTS (sorted by score):")
print(f"{'='*70}")
for r in tuning_results["video"]:
    print(f"  {r['score']:.4f}  ep={r['epoch']}  {r['name']}  params={r['params']}")

if tuning_results["video"]:
    best_video = tuning_results["video"][0]
    print(f"\nBEST VIDEO: {best_video['score']:.4f} — {best_video['params']}")
else:
    print("\nNo successful video trials")

### 0c) Apply best tuning results to full-training config

This cell updates the pipeline config with the best hyperparameters found above, then rewrites the config JSON for the 100-epoch runs. Fusion tuning runs *after* audio + video training (needs their checkpoints).

In [ ]:
# ── Apply best audio params ──
if tuning_results["audio"]:
    ba = tuning_results["audio"][0]["params"]
    cfg["audio"]["training"]["lr"]            = ba["lr"]
    cfg["audio"]["training"]["weight_decay"]  = ba["weight_decay"]
    cfg["audio"]["model"]["dropout"]          = ba["dropout"]
    cfg["audio"]["training"]["sequence_mil"]["class_weight_power"]     = ba["cwp"]
    cfg["audio"]["training"]["sequence_mil"]["balanced_sampler_power"] = ba["bsp"]
    cfg["audio"]["training"]["sequence_mil"]["topk_ratio_pos"]        = ba["topk_pos"]
    cfg["audio"]["training"]["sequence_mil"]["eval_pool_ratio"]       = ba["topk_pos"]
    print(f"Applied best AUDIO params (tuning score {tuning_results['audio'][0]['score']:.4f}):")
    print(f"  lr={ba['lr']:.2e}  wd={ba['weight_decay']:.2e}  dropout={ba['dropout']}")
    print(f"  cwp={ba['cwp']:.3f}  bsp={ba['bsp']:.3f}  topk_pos={ba['topk_pos']}")
else:
    print("No audio tuning results — keeping default params")

# ── Apply best video params ──
if tuning_results["video"]:
    bv = tuning_results["video"][0]["params"]
    cfg["video_window"]["training"]["lr"]                     = bv["lr"]
    cfg["video_window"]["training"]["weight_decay"]           = bv["weight_decay"]
    cfg["video_window"]["model"]["dropout"]                   = bv["dropout"]
    cfg["video_window"]["training"]["class_weight_power"]     = bv["cwp"]
    cfg["video_window"]["training"]["balanced_sampler_power"] = bv["bsp"]
    print(f"\nApplied best VIDEO params (tuning score {tuning_results['video'][0]['score']:.4f}):")
    print(f"  lr={bv['lr']:.2e}  wd={bv['weight_decay']:.2e}  dropout={bv['dropout']}")
    print(f"  cwp={bv['cwp']:.3f}  bsp={bv['bsp']:.3f}")
else:
    print("\nNo video tuning results — keeping default params")

# ── Restore full-training settings ──
cfg["audio"]["training"]["num_epochs"]       = 100
cfg["audio"]["training"]["patience"]         = 30
cfg["audio"]["training"]["checkpoint_dir"]   = "checkpoints/audio_best_100"
cfg["video_window"]["training"]["epochs"]    = 100
cfg["video_window"]["training"]["patience"]  = 30
cfg["video_window"]["training"]["checkpoint_dir"]   = "checkpoints/video_best_100"
cfg["video_window"]["training"]["checkpoint_path"]  = "checkpoints/video_best_100/window_classifier.pth"

# ── Write updated config ──
run_cfg_path.write_text(json.dumps(cfg, indent=2))
print(f"\nUpdated config written: {run_cfg_path}")

## 1) Train audio model (100 epochs)

In [5]:
run_cmd([
    sys.executable, '-u', '-m', 'audio.run_audio',
    '--config', str(run_cfg_path),
])

assert audio_ckpt.exists(), f"Audio checkpoint not found: {audio_ckpt}"
print(f"Audio best checkpoint: {audio_ckpt}")


RUN: /home/alolli/miniconda3/envs/therness_env/bin/python -u -m audio.run_audio --config /home/alolli/src/malto/hackathon/therness-hackaton-2026-polito/configs/master_config.best_100_pipeline.json
Device: cuda

Total weld files: 1551
File label distribution (multiclass): {'excessive_penetration': 259, 'good_weld': 731, 'excessive_convexity': 80, 'lack_of_fusion': 158, 'crater_cracks': 75, 'burnthrough': 169, 'overlap': 79}
Split strategy: stratified
Train welds: 1240 | Val welds: 311
File split stats | train={ burnthrough: 135 (10.9%), crater_cracks: 60 (4.8%), excessive_convexity: 64 (5.2%), excessive_penetration: 207 (16.7%), good_weld: 585 (47.2%), lack_of_fusion: 126 (10.2%), overlap: 63 (5.1%) } | val={ burnthrough: 34 (10.9%), crater_cracks: 15 (4.8%), excessive_convexity: 16 (5.1%), excessive_penetration: 52 (16.7%), good_weld: 146 (46.9%), lack_of_fusion: 32 (10.3%), overlap: 16 (5.1%) }
Train files: 1240 | Val files: 311
Classes (7): {'burnthrough': 0, 'crater_cracks': 1, 'ex

Training MIL multiclass:   0%|          | 0/155 [00:00<?, ?it/s]                     

Train loss: 1.7547 | Val loss: 1.7732 | Val Macro F1: 0.1156 | LR: 7e-06
New best model saved (val_f1=0.1156)

Epoch 2/100
----------------------------------------


Training MIL multiclass:   0%|          | 0/155 [00:00<?, ?it/s]                     

Train loss: 1.6877 | Val loss: 1.7000 | Val Macro F1: 0.1094 | LR: 1.4e-05
No improvement for 1 epoch(s)

Epoch 3/100
----------------------------------------


...

Training MIL multiclass:   0%|          | 0/155 [00:00<?, ?it/s]                      

Train loss: 0.1086 | Val loss: 0.1988 | Val Macro F1: 0.8686 | LR: 1e-06
No improvement for 14 epoch(s)

Epoch 99/100
----------------------------------------


Training MIL multiclass:   0%|          | 0/155 [00:00<?, ?it/s]                      

Train loss: 0.1108 | Val loss: 0.1985 | Val Macro F1: 0.8402 | LR: 1e-06
No improvement for 15 epoch(s)

Epoch 100/100
----------------------------------------


Train loss: 0.1108 | Val loss: 0.1968 | Val Macro F1: 0.8577 | LR: 1e-06
No improvement for 16 epoch(s)

Training complete. Best epoch: 84 (val_f1=0.9055)

Best epoch: 84
Train losses: ['1.7547', '1.6877', '1.5610', '1.3687', '1.0947', '0.8499', '0.7004', '0.6038', '0.5415', '0.5558', '0.4987', '0.4785', '0.5123', '0.5028', '0.3837', '0.3647', '0.3554', '0.3338', '0.2156', '0.1770', '0.1138', '0.0950', '0.1013', '0.0951', '0.0998', '0.0976', '0.1024', '0.0987', '0.1043', '0.1007', '0.0921', '0.1108', '0.1342', '0.1589', '0.1852', '0.2260', '0.1978', '0.2233', '0.2139', '0.1950', '0.1913', '0.1614', '0.1422', '0.1420', '0.1403', '0.1417', '0.1490', '0.1385', '0.1440', '0.1465', '0.1272', '0.1330', '0.1338', '0.1446', '0.1485', '0.1437', '0.1476', '0.1484', '0.1330', '0.1260', '0.1192', '0.1096', '0.1077', '0.1017', '0.1049', '0.0985', '0.0953', '0.0969', '0.0911', '0.0922', '0.0881', '0.0917', '0.0952', '0.0906', '0.0921', '0.0927', '0.0932', '0.0950', '0.0982', '0.0983', '0.0989', '0.1

## 2) Train video model (100 epochs)

In [ ]:
run_cmd([
    sys.executable, '-u', '-m', 'video.run_video',
    '--config', str(run_cfg_path),
])

assert video_ckpt.exists(), f"Video checkpoint not found: {video_ckpt}"
print(f"Video best checkpoint: {video_ckpt}")

### 0d) Fusion tuning (requires audio + video checkpoints from above)

In [ ]:
_rnd.seed(456)

assert audio_ckpt.exists(), f"Audio checkpoint needed for fusion tuning: {audio_ckpt}"
assert video_ckpt.exists(), f"Video checkpoint needed for fusion tuning: {video_ckpt}"

for i in range(N_FUSION_TRIALS):
    trial_params = {
        "lr":           _sample_uniform(5e-5, 2e-4, log=True),
        "weight_decay": _sample_uniform(5e-5, 2e-4, log=True),
        "dropout":      _sample_choice([0.15, 0.2, 0.25, 0.3]),
        "hidden_dim":   _sample_choice([128, 160, 192, 256]),
        "temporal_layers": _sample_choice([1, 2]),
    }
    trial_name = (
        f"fusion_tune_{i:02d}_lr={trial_params['lr']:.1e}"
        f"_hd={trial_params['hidden_dim']}"
        f"_do={trial_params['dropout']}"
    )
    overrides = {
        "fusion.training.num_epochs":       TUNING_EPOCHS,
        "fusion.training.patience":         TUNING_PATIENCE,
        "fusion.training.lr":               trial_params["lr"],
        "fusion.training.weight_decay":     trial_params["weight_decay"],
        "fusion.model.dropout":             trial_params["dropout"],
        "fusion.model.hidden_dim":          trial_params["hidden_dim"],
        "fusion.model.temporal_layers":     trial_params["temporal_layers"],
        "fusion.training.checkpoint_dir":   f"checkpoints/_tuning/{trial_name}",
    }
    result = run_trial(
        "fusion.run_fusion", overrides, trial_params, trial_name,
        extra_args=["--audio_checkpoint", str(audio_ckpt),
                     "--video_checkpoint", str(video_ckpt)],
    )
    if result:
        tuning_results["fusion"].append(result)

tuning_results["fusion"].sort(key=lambda r: r["score"], reverse=True)
print(f"\n{'='*70}")
print("FUSION TUNING RESULTS (sorted by score):")
print(f"{'='*70}")
for r in tuning_results["fusion"]:
    print(f"  {r['score']:.4f}  ep={r['epoch']}  {r['name']}  params={r['params']}")

if tuning_results["fusion"]:
    bf = tuning_results["fusion"][0]["params"]
    cfg["fusion"]["training"]["lr"]           = bf["lr"]
    cfg["fusion"]["training"]["weight_decay"] = bf["weight_decay"]
    cfg["fusion"]["model"]["dropout"]         = bf["dropout"]
    cfg["fusion"]["model"]["hidden_dim"]      = bf["hidden_dim"]
    cfg["fusion"]["model"]["temporal_layers"] = bf["temporal_layers"]
    # Restore full-training settings
    cfg["fusion"]["training"]["num_epochs"]     = 100
    cfg["fusion"]["training"]["patience"]       = 30
    cfg["fusion"]["training"]["checkpoint_dir"] = "checkpoints/fusion_best_100"
    run_cfg_path.write_text(json.dumps(cfg, indent=2))
    print(f"\nApplied best FUSION params (tuning score {tuning_results['fusion'][0]['score']:.4f}):")
    print(f"  lr={bf['lr']:.2e}  wd={bf['weight_decay']:.2e}  dropout={bf['dropout']}")
    print(f"  hidden_dim={bf['hidden_dim']}  temporal_layers={bf['temporal_layers']}")
    print(f"Updated config: {run_cfg_path}")
else:
    print("\nNo successful fusion trials — keeping default params")

## 3) Train fusion model (100 epochs) with best audio+video and tuned params

In [ ]:
run_cmd([
    sys.executable, '-u', '-m', 'fusion.run_fusion',
    '--config', str(run_cfg_path),
    '--audio_checkpoint', str(audio_ckpt),
    '--video_checkpoint', str(video_ckpt),
])

assert fusion_ckpt.exists(), f"Fusion checkpoint not found: {fusion_ckpt}"
print(f"Fusion best checkpoint: {fusion_ckpt}")

## 4) Optional: evaluate best fusion checkpoint

In [ ]:
run_cmd([
    sys.executable, '-u', '-m', 'fusion.run_fusion',
    '--config', str(run_cfg_path),
    '--audio_checkpoint', str(audio_ckpt),
    '--video_checkpoint', str(video_ckpt),
    '--test_only',
    '--checkpoint', str(fusion_ckpt),
])